In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/content/jupyter/mta-accessibility')
from src.data import process_turnstile

In [2]:
process_turnstile.clean_turnstile_data()

/content/jupyter/mta-accessibility/src/data/process_turnstile.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  turnstile = pd.concat([t_jan_nov,t_nov_dec])


In [3]:
tmp = pd.read_pickle('../../data/processed/cleaned_turnstile_data_2019.pkl.gz')

In [4]:
tmp.head()

,datetime,C/A,DESC,DIVISION,ENTRIES,EXITS,LINENAME,SCP,STATION,UNIT,entry_diff,exit_diff,entry_diff_abs,exit_diff_abs,time_diffs
86198,2019-01-01,N095,REGULAR,IND,1179754,746536,ACJZ2345,00-03-07,FULTON ST,R014,22.0,58.0,22.0,58.0,4.0
86199,2019-01-01,R639,REGULAR,IRT,21930687,2308213,25,00-00-03,CHURCH AV,R109,271.0,68.0,271.0,68.0,4.0
86200,2019-01-01,N530,REGULAR,IND,14953428,7013073,F,00-00-01,YORK ST,R301,159.0,344.0,159.0,344.0,4.0
86201,2019-01-01,R639,REGULAR,IRT,50333402,16777216,25,00-05-00,CHURCH AV,R109,0.0,0.0,0.0,0.0,4.0
86202,2019-01-01,N120A,REGULAR,IND,2901395,1599171,AC,01-00-00,UTICA AV,R153,125.0,226.0,125.0,226.0,4.0
